In [1]:
import random

In [ ]:
def recreate(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    for c in solution[1]:
        pos_best = None
        makespan_best = float('inf')
        
        feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
        for pos in feasible_truck_tour_positions: 
            makespan_diff = completion_time(instance, insert_customer(c, pos, solution)) - makespan(instance, solution)
            if pos_best == None or ((pos_best != None and makespan_diff <= makespan_best) and random.random() > (1 - gamma)):
                makespan_best = makespan_diff
                pos_best = pos
        
        if drone_eligible(isntance, solution[0][1], c):
            chosen_drone = largest_time_slack_drone(solution[0][1])
            makespan_diff = completion_time(instance, insert_customer(c, [1, chosen_drone, 0], solution)) - makespan(instance, solution)
            if pos_best == None or ((pos_best != None and makespan_diff <= makespan_best) and random.random() > (1 - gamma)):
                makespan_best = makespan_diff
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            solution[0][0].append([])
            pos_best[0, len(solution[0][0]) - 1, 0]
        
        solution = insert_customer(c, pos_best, solution)
        solution[1].remove(c)

    return solution

In [ ]:
def SISSRs_min_time(instance, delta, epsilon, iter_max):
    s_0 = initial_solution_construction(instance)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    while (terminal_condition_met() == False):
        s = ruin_and_recreate(instance, s_curr)
        if makespan(instance, s) < makespan(instance, s_curr)*(1+delta) or (makespan(instance, s) == makespan(instance, s_curr) and total_completion_time(instance,s) < total_completion_time(instance, s_curr)):
            s_curr = local_search(instance, s)
            if makespan(instance, s_curr) < makespan(instance, s_best) or (makespan(instance, s) == makespan(instance, s_best) and total_completion_time(instance,s) < total_completion_time(instance, s_best)):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_max:
            s_curr = perturbate(instance, s_curr)
            iterations_without_improvement = 0
        delta = delta * epsilon

    return s_best